In [391]:
import sys
import os
import string
from glob import glob

# Create by index of symbol folder in the train and test folders

In [392]:
ALL_SYMBOLS = string.ascii_uppercase + string.digits + "!"

In [393]:
target_mapper = {}
for index, value in enumerate(ALL_SYMBOLS):
    target_mapper[value] = index

In [394]:
target_mapper["A"], target_mapper["Z"]

(0, 25)

In [395]:
parent_dir = os.getcwd()

In [396]:
train_data = os.path.join(parent_dir, "training_data")
test_data = os.path.join(parent_dir, "test_data")

In [397]:
for symbol in target_mapper.keys():
    path = os.path.join(train_data, str(target_mapper[symbol]))
    try:
        os.mkdir(path)
    except:
        print(f"{path} already exists")

    path = os.path.join(test_data, str(target_mapper[symbol]))
    try:
        os.mkdir(path)
    except:
        print(f"{path} already exists")

# Split every image, generated by captcha into the either training folder or testing

In [398]:
import cv2 as cv
import random
import re
import numpy as np

In [399]:
def get_min(val1, val2):
    res = val2
    if val1 < val2:
        res = val1
    return res

def get_max(val1, val2):
    res = val2
    if val1 > val2:
        res = val1
    return res

In [400]:
def get_corners(contour):
    min_x = contour[0][0][0]
    min_y = contour[0][0][1]
    max_x = contour[0][0][0]
    max_y = contour[0][0][1]
    for point in contour:
        x, y = point[0]
        min_x = get_min(min_x, x)
        min_y = get_min(min_y, y)
        max_x = get_max(max_x, x)
        max_y = get_max(max_y, y)

    return ((min_x, min_y), (max_x, max_y))

In [401]:
def get_square(start, end):
    length = end[0] - start[0]
    height = end[1] - start[1]
    return length * height

In [402]:
def intersect(boxA, boxB):
    startA, endA = boxA
    startB, endB = boxB

    flag_intersect = False
    if (startA[1] > startB[1]) and (startA[1] < endB[1]) and (startA[0] > startB[0]) and (startA[0] < endB[0]):
        flag_intersect = True
    if (startB[1] > startA[1]) and (startB[1] < endA[1]) and (startB[0] > startA[0]) and (startB[0] < endA[0]):
        flag_intersect = True
    return flag_intersect

In [403]:
def filter_by_intersection(boxes):
    dct = {}
    for box in boxes:
        if box not in dct:
            dct[box] = 0
        for box2 in boxes:
            if box2 != box:
                if intersect(box, box2):
                    dct[box] += 1
    result = [key[0] for key in filter(lambda x: x[1] < 3, dct.items())]
    return result

In [404]:
def increase_14px(box):
    start, end = box
    start = (start[0] - 7, start[1] - 7)
    end = (end[0] + 7, end[1] + 7)
    return start, end

In [405]:
def inner_contour(boxA, boxB):
    startA, endA = boxA
    startB, endB = boxB
    start_lower = (startA[0] <= startB[0]) and (startA[1] <= startB[1])
    end_bigger = (endA[0] >= endB[0]) and (endA[1] >= endB[1])
    return start_lower and end_bigger

In [406]:
def filter_inner_contour(boxes):
    result = []
    for i in range(len(boxes)):
        flag = False
        for j in range(len(boxes)):
            if j != i:
                if inner_contour(boxes[i], boxes[j]):
                    flag = True
        if not flag:
            result.append(boxes[i])
    return result

In [407]:
def split_rectangles(boxes):
    boxes = list(sorted(boxes, key=lambda x: x[1][0] - x[0][0]))
    box_to_split = boxes.pop()
    start, end = box_to_split
    width = end[0] - start[0]
    height = end[1] - start[1]
    box1 = ((start[0], start[1]), (start[0] + (width // 2), end[1]))
    box2 = ((start[0] + (width // 2), start[1]), (end[0], end[1]))
    boxes.append(box1)
    boxes.append(box2)
    return boxes

For Rectangle1:

x_distance = abs(L1.x – R1.x) 
y_distance = abs(L1.y – R1.y) 
Area of Rectangle1 = x_distance * y_distance
For Rectangle2:

x_distance = abs(L2.x – R2.x) 
y_distance = abs(L2.y – R2.y)
Area of Rectangle2 = x_distance * y_distance
For area of overlapping Rectangle:

x_distance = min(R1.x, R2.x) – max(L1.x, L2.x)
y_distance = min(R1.y, R2.y) – max(L1.y, L2.y)
Area of overlapping rectange = x_distance * y_distance
If the x_distance or y_distance is negative, then the two rectangles do not intersect. In that case, overlapping area is 0.

In [408]:
def intersect_area(box1, box2):
    start1, end1 = box1
    start2, end2 = box2
    x_distance = min(end1[0], end2[0]) - max(start1[0], start2[0])
    y_distance = min(end1[1], end2[1]) - max(start1[1], start2[1])
    return y_distance * x_distance

In [409]:
def join_two_rects(box1, box2):
    start1, end1 = box1
    start2, end2 = box2
    start = (min(start1[0], start2[0]), min(start1[1], start2[1]))
    end = (max(end1[0], end2[0]), min(end1[1], end2[1]))
    return (start, end)

In [410]:
def filter_by_biggest_interarea(boxes):
    dct = {}
    for i in range(len(boxes)):
        for j in range(len(boxes)):
            if j > i:
                dct[(i, j)] = intersect_area(boxes[i], boxes[j])
    keys = list(sorted(dct.keys(), key=lambda x: dct[x]))
    box2 = boxes.pop(keys[-1][1])
    box1 = boxes.pop(keys[-1][0])
    boxes.append(join_two_rects(box1, box2))
    return boxes

In [411]:
all_captchas = glob("captcha_images/*.png")

In [412]:
sampleList = ["training_data", "test_data"]
 
randomList = random.choices(sampleList, weights=(80, 20), k=len(all_captchas))

In [413]:
idx = 10
all_captchas[idx]

'captcha_images\\01UZ.png'

In [414]:
# rewrite join

In [415]:
"{0:010d}.png".format(5)

'0000000005.png'

In [416]:
curr_symb = 0
for idx in range(len(all_captchas)):
    if randomList[idx]=="training_data":
        path = train_data
    else:
        path = test_data
        
    img = cv.imread(all_captchas[idx])
    
    # Convert the image to grayscale
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_copy = img.copy()
    
    img = cv.bilateralFilter(img,9,75,75)
    
    threshold = 75
    
    # Detect edges using Canny
    canny_output = cv.Canny(img, threshold, threshold * 2)
    # img = cv.adaptiveThreshold(img_copy, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 15, -2)
    
    # Display the result
    # cv.imshow('Result', canny_output)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    
    # Find contours
    contours, hierarchy = cv.findContours(canny_output, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    # get rectangles
    contours = [get_corners(i) for i in contours]
    contours = list(filter(lambda x: get_square(*x) > 400, contours))
    contours = filter_inner_contour(contours)
    contours = sorted(contours, key=lambda x: get_square(*x))
    contours = [increase_14px(i) for i in contours]
    if not contours:
        continue
    if len(contours) < 4:
        contours = split_rectangles(contours)
    if len(contours) > 4:
        contours = filter_by_biggest_interarea(contours)
    contours = filter_by_intersection(contours)

    symbols = re.search(r"\\(\w+)\.", all_captchas[idx]).group(1)
    # Draw contours
    # drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    for i in range(len(contours)):
        color = (random.randint(0,256), random.randint(0,256), random.randint(0,256))
        # img = cv.rectangle(img, contours[i][0], contours[i][1], color, 5)
        try:
            symbol = symbols[i]
        except:
            pass
        curr_path = os.path.join(path, symbol)
        curr_path = os.path.join(curr_path, "{0:010d}.png".format(curr_symb))
        curr_symb += 1
        crop_img = img[contours[i][0][1]:contours[i][1][1], contours[i][0][0]:contours[i][1][0]]

        if crop_img.shape[0] == 0:
            continue
        if crop_img.shape[1] == 0:
            continue
        cv.imwrite(curr_path, crop_img)

    # break
        
    # Display the result
    # cv.imshow('Result', img)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

# Model creation

In [374]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import keras
import tensorflow as tf
from keras import backend as K
from keras import metrics
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.metrics import AUC
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

In [375]:
seed_value= 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED']=str(seed_value)

In [376]:
train_path = os.path.join(os.getcwd(), "training_data")
test_path = os.path.join(os.getcwd(), "test_data")

In [377]:
## Set up hyperparameters that will be used later
hyper_dimension = 64
hyper_batch_size = 128
hyper_epochs = 200
hyper_channels = 1
hyper_mode = 'grayscale'

In [378]:
## Generate batches of image data (train, validation, and test) with data augmentation
## Augmentation helps to lower variance of complex model
train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 

In [379]:
train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    target_size = (hyper_dimension, hyper_dimension),
                                                    batch_size = hyper_batch_size, 
                                                    color_mode = hyper_mode,
                                                    class_mode = 'categorical', 
                                                    seed = seed_value)
test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'categorical',
                                                 color_mode = hyper_mode,
                                                 shuffle=False,
                                                 seed = seed_value)

test_generator.reset()

Found 7943 images belonging to 37 classes.
Found 1937 images belonging to 37 classes.


In [380]:
train_generator.image_shape

(64, 64, 1)

In [385]:
# Build the neural network!
model = Sequential()

# First convolutional layer with max pooling
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(64, 64, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Second convolutional layer with max pooling
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Hidden layer with 500 nodes
model.add(Flatten())
model.add(Dense(500, activation="relu"))

# Output layer with 36 nodes (one for each possible letter/number we predict)
model.add(Dense(37, activation="softmax"))

# Ask Keras to build the TensorFlow model behind the scenes
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [386]:
hist = model.fit(
    train_generator, 
    steps_per_epoch = len(train_generator), 
    epochs = hyper_epochs, 
    validation_data = test_generator,
    validation_steps = len(test_generator), 
    verbose=2
)

Epoch 1/200
63/63 - 21s - 328ms/step - accuracy: 0.1035 - loss: 2.3840 - val_accuracy: 0.1038 - val_loss: 2.3847
Epoch 2/200
63/63 - 0s - 536us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/200
63/63 - 16s - 255ms/step - accuracy: 0.1035 - loss: 2.3199 - val_accuracy: 0.0950 - val_loss: 2.3054
Epoch 4/200
63/63 - 0s - 384us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/200
63/63 - 15s - 246ms/step - accuracy: 0.1078 - loss: 2.3129 - val_accuracy: 0.1239 - val_loss: 2.3058
Epoch 6/200
63/63 - 0s - 303us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/200
63/63 - 15s - 245ms/step - accuracy: 0.1123 - loss: 2.3098 - val_accuracy: 0.1275 - val_loss: 2.2940
Epoch 8/200
63/63 - 0s - 501us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/200
63/63 - 20s - 319ms/step

# Save the model to binary

In [387]:
model.save('cnn_captcha.h5')

# Prediction of model

In [388]:
target_mapper = {}
for index, value in enumerate(ALL_SYMBOLS):
    target_mapper[str(index)] = value